### 바디럽 리뷰 수집

In [1]:
# 페이지에 요청하기 위한 용도
import requests as req
# html 데이터를 만들기 위한 용도
from bs4 import BeautifulSoup as bs

In [3]:
# 바디럽 페이지 정보 요청하기
url = 'https://bodyluv.kr/product/%EB%B0%94%EB%94%94%EB%9F%BD-%EB%94%A5%EC%8A%AC%EB%A6%BD-%EC%BF%A8-%EC%9D%B4%EB%B6%88-v2-%EC%95%88%ED%8B%B0%EB%B2%84%EA%B7%B8/437/category/1/display/2/#prdReview'
h = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
res = req.get(url,headers = h)
res

<Response [200]>

In [5]:
# 요청받은 정보 중에서 html정보 가져오기
res.text # 문자열 형식으로 가져와졌음
soup = bs(res.text, 'lxml') # html 데이터로 변경하기

In [7]:
# review_list_v2__message js-translate-text
# 리뷰를 수집할 때 많이 발생 
# > 한 페이지 안에서 원하는 값이 여러개의 페이지로 구성되어있을 때 많이 발생
# > 새로고침할 때 마다 많은 자원이 소모된다 > 하나의 페이지로 만들어서 보여주기만 하자
soup.select('div.review_list_v2__message.js-translate-text')

[]

In [17]:
# iframe : 기존 페이지안에 새로운 페이지를 삽입하는 태그
# 기존 페이지에서는 iframe안에 있는 값을 가져올 수 없음
# iframe 안에 있는 src 속성의 url을 접근해야지 리뷰를 가져올 수 있음
url = 'https://review4.cre.ma/bodyluv.kr/products/reviews?product_code=437&iframe_id=crema-product-reviews-2&widget_style=&app=0&parent_url=https%3A%2F%2Fbodyluv.kr%2Fproduct%2F%25EB%25B0%2594%25EB%2594%2594%25EB%259F%25BD-%25EB%2594%25A5%25EC%258A%25AC%25EB%25A6%25BD-%25EC%25BF%25A8-%25EC%259D%25B4%25EB%25B6%2588-v2-%25EC%2595%2588%25ED%258B%25B0%25EB%25B2%2584%25EA%25B7%25B8%2F437%2Fcategory%2F1%2Fdisplay%2F2%2F%23prdReview&nonmember_token=&secure_device_token=V2735b8a66cd83ed22d4d2bc8e296bd40d4f5cbe6021ae16a425192a9f299d5bbdb72cb385dafbbd050afb979c92b5e20f&iframe=1'
res = req.get(url)
soup = bs(res.text,'lxml')

In [22]:
# 첫번째 페이지의 리뷰 전부 수집하기
body = soup.select('div.review_list_v2__message.js-collapsed-review-content.js-translate-text')
len(body)

5

In [26]:
# 이스케이프 코드 : \n(개행), \t(탭), \', \\, \"
# strip : 문자열의 맨 앞과 맨 뒤의 공백 제거
body[0].text.strip()

'아이들 여름 이불을 바꾸려고 겅색중 알게되어 구입했는데 신랑이 탐을 내어 하나더 구입 했네요. 촉감도 양면으로 부트러운면콰 시윈한 냉감이 있어 좋네요. 여름 이불로 적극 추천입니다(2020-08-07 14:26:40 에 등록된 네이버 페이 구매평)'

In [28]:
cnt = 1
for review in body :
    # 리뷰 구분용 인덱스
    print(cnt)
    print(review.text.strip())
    cnt += 1

1
아이들 여름 이불을 바꾸려고 겅색중 알게되어 구입했는데 신랑이 탐을 내어 하나더 구입 했네요. 촉감도 양면으로 부트러운면콰 시윈한 냉감이 있어 좋네요. 여름 이불로 적극 추천입니다(2020-08-07 14:26:40 에 등록된 네이버 페이 구매평)
2
주말부부라 신랑이 원룸에서 사용중인데너무 좋다고아들것도 주문해주라고 난리랍니다..평소 불면증을 호소하며..힘들어했는데요즘은 그런 불평이 사라졌어요^^
3
신랑이 피부가 예민해서 피부과 약을 장기적으로 복용즁이에요  다른 이불은 피부가 긁혀 힘들어 했는데 바디럽 쓰고는 신세계라고 해여~ 감사해용! 제품 굿 세상 시원합니다!!
4
아이보리했구요 너무 화이트 아니라서 좋아요 아늑한 느낌이구요 배게커버도 양면이 달라서 좋고 특히 매트는 시원해요 요즘 잠을 제대로 못자는데 이거 깔고 잘잤어요 추천 네이버에 사진리뷰 처음올림 알려드리고싶어서(2022-06-25 13:05:51 에 등록된 네이버 페이 구매평)
5
아이보리 싱글이 품절이라 기다렸다가 드디어 받았어요!!왜 진작 안샀을까 싶을만큼 매트와 이불 만족도 넘 높아요.이불 너무 부드럽고 가벼워요.매트는 방수가 되서 물을 쏟아도 걱정없구요, 땀 흘려도 몸에 붙지 않아 좋아요.마약 베개도 구매하러 갑니다ㅎㅎ


### 1~9 페이지 리뷰 수집하기

In [36]:
# 1~9 페이지 리뷰 수집하기
# 페이지에 접근하기
cnt = 1
for page in range(1,10):
    url=f'https://review4.cre.ma/bodyluv.kr/products/reviews?app=0&iframe=1&iframe_id=crema-product-reviews-2&page={page}&parent_url=https%3A%2F%2Fbodyluv.kr%2Fproduct%2F%25EB%25B0%2594%25EB%2594%2594%25EB%259F%25BD-%25EB%2594%25A5%25EC%258A%25AC%25EB%25A6%25BD-%25EC%25BF%25A8-%25EC%259D%25B4%25EB%25B6%2588-v2-%25EC%2595%2588%25ED%258B%25B0%25EB%25B2%2584%25EA%25B7%25B8%2F437%2Fcategory%2F1%2Fdisplay%2F2%2F%23prdReview&product_code=437&secure_device_token=V295459948fbb81cbd3f5603256a8784336d37a1a8c455efa8f27475ffa29eb50eff71695b93599c0cef61f5f5888040fa&widget_env=100&widget_style='
    res = req.get(url) # html 정보 가져오기
    
    soup = bs(res.text,'lxml') # 문자열 html을 html데이터로 변경하기
    
    # 리뷰 수집하기(5개)
    body = soup.select('div.review_list_v2__message.js-collapsed-review-content.js-translate-text')
    # 리뷰 출력하기
    for review in body :
        # 리뷰 구분용 인덱스
        print(cnt)
        print(review.text.strip())
        cnt += 1

1
아이들 여름 이불을 바꾸려고 겅색중 알게되어 구입했는데 신랑이 탐을 내어 하나더 구입 했네요. 촉감도 양면으로 부트러운면콰 시윈한 냉감이 있어 좋네요. 여름 이불로 적극 추천입니다(2020-08-07 14:26:40 에 등록된 네이버 페이 구매평)
2
주말부부라 신랑이 원룸에서 사용중인데너무 좋다고아들것도 주문해주라고 난리랍니다..평소 불면증을 호소하며..힘들어했는데요즘은 그런 불평이 사라졌어요^^
3
신랑이 피부가 예민해서 피부과 약을 장기적으로 복용즁이에요  다른 이불은 피부가 긁혀 힘들어 했는데 바디럽 쓰고는 신세계라고 해여~ 감사해용! 제품 굿 세상 시원합니다!!
4
아이보리했구요 너무 화이트 아니라서 좋아요 아늑한 느낌이구요 배게커버도 양면이 달라서 좋고 특히 매트는 시원해요 요즘 잠을 제대로 못자는데 이거 깔고 잘잤어요 추천 네이버에 사진리뷰 처음올림 알려드리고싶어서(2022-06-25 13:05:51 에 등록된 네이버 페이 구매평)
5
아이보리 싱글이 품절이라 기다렸다가 드디어 받았어요!!왜 진작 안샀을까 싶을만큼 매트와 이불 만족도 넘 높아요.이불 너무 부드럽고 가벼워요.매트는 방수가 되서 물을 쏟아도 걱정없구요, 땀 흘려도 몸에 붙지 않아 좋아요.마약 베개도 구매하러 갑니다ㅎㅎ
6
배송 빨라서 바로 받았어요새탁하고 처음 깔아보았어요촉감이 정말 부드럽고 차가운 느낌이 있어서 첫느낌만으로도 매트리스와 가족들 이불도 고민되네요. ^^
7
양쪽을 다르천으로 만드신거 대박~ 내가좋아하는 느낌쪽을 베고 자면 굿~~(2021-07-03 01:14:04 에 등록된 네이버 페이 구매평)
8
아이가땀이많은아이인데우선누웠을때너무시원하다고좋아하고,촉감이부드러워서너무좋아해요가벼우면서도 선풍기바람에쉽게날리지않아서배가아플일도없어서좋네요
9
부드럽게 감기는 이불 시원한 느낌의 매트괜찮은거 같내요
10
바디럽에서 맨처음마약 토퍼, 쿨이불, 샤워기 등등 이렇게 구매해서 잘 사용하고 있었는데이번 여름 너무 더워 견딜 수 없어쿨 패드도 구매해봤어요!!기존에 쿨이불이 그레이 색

### 끝 페이지까지 리뷰 수집하기
- try-except

In [45]:
# 페이지에 접근하기
cnt = 1
# 몇 페이지가 끝인지 모르는 상황에서 끝까지 가보기
# Python : try-except
# Java : try-catch
page = 0
try :
    for page in [1,2,3,4,5,6,'a']:
        # 1. 1000페이지까지 이동한다.
        # 2. 중간에 페이지 끝을 마주한다.
        # 2.1 만약 페이지에 끝을 본다면 거기서 리뷰수집 마무리하고 다음으로 넘어가기
        url=f'https://review4.cre.ma/bodyluv.kr/products/reviews?app=0&iframe=1&iframe_id=crema-product-reviews-2&page={page}&parent_url=https%3A%2F%2Fbodyluv.kr%2Fproduct%2F%25EB%25B0%2594%25EB%2594%2594%25EB%259F%25BD-%25EB%2594%25A5%25EC%258A%25AC%25EB%25A6%25BD-%25EC%25BF%25A8-%25EC%259D%25B4%25EB%25B6%2588-v2-%25EC%2595%2588%25ED%258B%25B0%25EB%25B2%2584%25EA%25B7%25B8%2F437%2Fcategory%2F1%2Fdisplay%2F2%2F%23prdReview&product_code=437&secure_device_token=V295459948fbb81cbd3f5603256a8784336d37a1a8c455efa8f27475ffa29eb50eff71695b93599c0cef61f5f5888040fa&widget_env=100&widget_style='
        res = req.get(url) # html 정보 가져오기

        soup = bs(res.text,'lxml') # 문자열 html을 html데이터로 변경하기

        # 리뷰 수집하기(5개)
        body = soup.select('div.review_list_v2__message.js-collapsed-review-content.js-translate-text')
        # 리뷰 출력하기
        for review in body :
            # 리뷰 구분용 인덱스
            print(cnt)
            print(review.text.strip())
            cnt += 1
except:
    # 몇 번째 페이지에서 오류가 났는가
    print(page,'페이지에서 오류') # 파이썬이어서 될 것. 자바에서는 안됨. 안되는 게 개념적으로 맞음.
    
print('아직 끝나지 않았음')

1
아이들 여름 이불을 바꾸려고 겅색중 알게되어 구입했는데 신랑이 탐을 내어 하나더 구입 했네요. 촉감도 양면으로 부트러운면콰 시윈한 냉감이 있어 좋네요. 여름 이불로 적극 추천입니다(2020-08-07 14:26:40 에 등록된 네이버 페이 구매평)
2
주말부부라 신랑이 원룸에서 사용중인데너무 좋다고아들것도 주문해주라고 난리랍니다..평소 불면증을 호소하며..힘들어했는데요즘은 그런 불평이 사라졌어요^^
3
신랑이 피부가 예민해서 피부과 약을 장기적으로 복용즁이에요  다른 이불은 피부가 긁혀 힘들어 했는데 바디럽 쓰고는 신세계라고 해여~ 감사해용! 제품 굿 세상 시원합니다!!
4
아이보리했구요 너무 화이트 아니라서 좋아요 아늑한 느낌이구요 배게커버도 양면이 달라서 좋고 특히 매트는 시원해요 요즘 잠을 제대로 못자는데 이거 깔고 잘잤어요 추천 네이버에 사진리뷰 처음올림 알려드리고싶어서(2022-06-25 13:05:51 에 등록된 네이버 페이 구매평)
5
아이보리 싱글이 품절이라 기다렸다가 드디어 받았어요!!왜 진작 안샀을까 싶을만큼 매트와 이불 만족도 넘 높아요.이불 너무 부드럽고 가벼워요.매트는 방수가 되서 물을 쏟아도 걱정없구요, 땀 흘려도 몸에 붙지 않아 좋아요.마약 베개도 구매하러 갑니다ㅎㅎ
6
배송 빨라서 바로 받았어요새탁하고 처음 깔아보았어요촉감이 정말 부드럽고 차가운 느낌이 있어서 첫느낌만으로도 매트리스와 가족들 이불도 고민되네요. ^^
7
양쪽을 다르천으로 만드신거 대박~ 내가좋아하는 느낌쪽을 베고 자면 굿~~(2021-07-03 01:14:04 에 등록된 네이버 페이 구매평)
8
아이가땀이많은아이인데우선누웠을때너무시원하다고좋아하고,촉감이부드러워서너무좋아해요가벼우면서도 선풍기바람에쉽게날리지않아서배가아플일도없어서좋네요
9
부드럽게 감기는 이불 시원한 느낌의 매트괜찮은거 같내요
10
바디럽에서 맨처음마약 토퍼, 쿨이불, 샤워기 등등 이렇게 구매해서 잘 사용하고 있었는데이번 여름 너무 더워 견딜 수 없어쿨 패드도 구매해봤어요!!기존에 쿨이불이 그레이 색